In [122]:
import pandas as pd
import nltk
import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer




In [123]:
df = pd.read_csv('multi_questions.csv')
df.head()

,QuestionID,QuestionType,Category,AskerID,QuestionTime,QuestionText
0,C1Q1,yes/no,Automotive,A365S8H55GGXPD,"July 19, 2013",will they fit 2013 f350 dually
1,C1Q2,yes/no,Automotive,AXOOEUYEJ87ZB,"May 7, 2014",will they fit 2014 mazda 3 sport?
2,C1Q3,open-ended,Automotive,AN2AMELSNPN99,"June 20, 2014",Do they fit a 1998 GMC Sierra 3 door?
3,C1Q4,yes/no,Automotive,A367QVRWPWFTLT,"September 3, 2013",will this work on 95 bonneville and 94 camaro
4,C1Q5,yes/no,Automotive,A15Q5XTXEKWK0E,"November 3, 2013",will this fit a 1996 4 winns 238 vista 5.0 w/v...


In [124]:
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
def stemming(message):
    message = message.split()
    stemmed = [lemmatizer.lemmatize(word) for word in message]
    stemmed = ' '.join(stemmed)
    return stemmed
df['QuestionText'] = df['QuestionText'].apply(stemming)
df.head()

,QuestionID,QuestionType,Category,AskerID,QuestionTime,QuestionText
0,C1Q1,yes/no,Automotive,A365S8H55GGXPD,"July 19, 2013",will they fit 2013 f350 dually
1,C1Q2,yes/no,Automotive,AXOOEUYEJ87ZB,"May 7, 2014",will they fit 2014 mazda 3 sport?
2,C1Q3,open-ended,Automotive,AN2AMELSNPN99,"June 20, 2014",Do they fit a 1998 GMC Sierra 3 door?
3,C1Q4,yes/no,Automotive,A367QVRWPWFTLT,"September 3, 2013",will this work on 95 bonneville and 94 camaro
4,C1Q5,yes/no,Automotive,A15Q5XTXEKWK0E,"November 3, 2013",will this fit a 1996 4 winns 238 vista 5.0 w/v...


In [125]:
english_stop_words = stopwords.words('english')

def cleaning(message):
    Test_punc_removed = [char for char in message if char not in string.punctuation]
    Test_punc_removed_join = ''.join(Test_punc_removed)
    Test_punc_removed_join=Test_punc_removed_join.split()
    Test_punc_removed_join_clean = [word+" " for word in Test_punc_removed_join if word.lower() not in english_stop_words and word.isalpha()]
    Test_punc_removed_join_Text = ' '.join(Test_punc_removed_join_clean)
    return Test_punc_removed_join_Text


df['QuestionText'] = df['QuestionText'].apply(cleaning)
df.head()

,QuestionID,QuestionType,Category,AskerID,QuestionTime,QuestionText
0,C1Q1,yes/no,Automotive,A365S8H55GGXPD,"July 19, 2013",fit dually
1,C1Q2,yes/no,Automotive,AXOOEUYEJ87ZB,"May 7, 2014",fit mazda sport
2,C1Q3,open-ended,Automotive,AN2AMELSNPN99,"June 20, 2014",fit GMC Sierra door
3,C1Q4,yes/no,Automotive,A367QVRWPWFTLT,"September 3, 2013",work bonneville camaro
4,C1Q5,yes/no,Automotive,A15Q5XTXEKWK0E,"November 3, 2013",fit winns vista wvolvo sx drive


In [126]:
from sklearn.feature_extraction.text import CountVectorizer
transformer = CountVectorizer(min_df=0.05, lowercase=True).fit(df['QuestionText'])
vocab = transformer.get_feature_names()
for i in vocab:
    x=[]
    for j in df["QuestionText"]:
        counter=0
        if i in j:
            x.append(j.count(i))
        else:
            x.append(0)
    df[i]=x
df['QuestionText'] = df['QuestionText'].apply(cleaning)
df.head()

,QuestionID,QuestionType,Category,AskerID,QuestionTime,QuestionText,come,doe,fit,use,work
0,C1Q1,yes/no,Automotive,A365S8H55GGXPD,"July 19, 2013",fit dually,0,0,1,0,0
1,C1Q2,yes/no,Automotive,AXOOEUYEJ87ZB,"May 7, 2014",fit mazda sport,0,0,1,0,0
2,C1Q3,open-ended,Automotive,AN2AMELSNPN99,"June 20, 2014",fit GMC Sierra door,0,0,1,0,0
3,C1Q4,yes/no,Automotive,A367QVRWPWFTLT,"September 3, 2013",work bonneville camaro,0,0,0,0,1
4,C1Q5,yes/no,Automotive,A15Q5XTXEKWK0E,"November 3, 2013",fit winns vista wvolvo sx drive,0,0,1,0,0


In [127]:
from sklearn.preprocessing import LabelEncoder  

le = LabelEncoder()
le2=LabelEncoder()

le2.fit_transform(df['QuestionType'])

df['QuestionType']=le2.fit_transform(df['QuestionType'])

In [128]:
df.drop(['QuestionID'],axis=1,inplace=True)
df.drop(["AskerID"],axis=1,inplace=True)
df.drop(["QuestionTime"],axis=1,inplace=True)
df.drop(["QuestionText"],axis=1,inplace=True)
df.drop(["Category"],axis=1,inplace=True)
df.head()

,QuestionType,come,doe,fit,use,work
0,1,0,0,1,0,0
1,1,0,0,1,0,0
2,0,0,0,1,0,0
3,1,0,0,0,0,1
4,1,0,0,1,0,0


In [129]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
# test train split
from sklearn.model_selection import train_test_split

test,train = train_test_split(df,test_size=0.2,random_state=42)

model_G =  GaussianMixture(2**len(vocab)).fit(train)
model_k = KMeans(2**len(vocab)).fit(train)

In [130]:
labels_G=model_G.predict(test)
labels_k=model_k.predict(test)
print("K means score :- ",silhouette_score(test,labels_k))

print("G means score :- ",silhouette_score(test,labels_G))

K means score :-  0.96184550846677
G means score :-  0.9562408750429281
